# 03: Runtime Local Testing Mode (No Cloud Credentials)

This notebook demonstrates IBM Runtime **local testing mode** using a fake backend, so you can debug a Runtime-style flow without cloud credentials.

What this demonstrates:
- Use `SamplerV2(mode=fake_backend)` in local testing mode.
- Keep the same transpile-plus-primitives shape used for cloud jobs.
- Print job metadata and sampled bitstrings locally.

In [ ]:
import qiskit
import qiskit_ibm_runtime
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

print('Qiskit version:', qiskit.__version__)
print('qiskit-ibm-runtime version:', qiskit_ibm_runtime.__version__)

In [ ]:
fake_backend = FakeManilaV2()
print('Fake backend:', fake_backend.name)
print('Qubits:', fake_backend.num_qubits)

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

pm = generate_preset_pass_manager(backend=fake_backend, optimization_level=1)
isa_qc = pm.run(qc)

In [ ]:
sampler = SamplerV2(mode=fake_backend, options={'simulator': {'seed_simulator': 1234}})
job = sampler.run([isa_qc], shots=256)

print('Local runtime job ID:', job.job_id())
print('Job status (submitted):', job.status())

pub_result = job.result()[0]
print('Job status (after result):', job.status())

counts = pub_result.data.meas.get_counts()
bitstrings = pub_result.data.meas.get_bitstrings()

print('Counts:', counts)
print('First 12 bitstrings:', bitstrings[:12])

Use this notebook as a no-credentials Runtime debugging step before running `notebooks/02_ibm_runtime_smoke.ipynb` against real hardware.